In [1]:
from typing import Optional, Any, List


class Graph:
    """
    Graph class
    """
    def __init__(self):
        self._graph = {}

    def add_vertex(self, vertex: str, data: Optional[Any]=None) -> None:
        """
        Adds a vertex to the graph
        :param vertex: the vertex name
        :param data: data associated with the vertex
        """
        if vertex not in self._graph:
            self._graph[vertex] = {'data': data, 'neighbors': {}}

    def add_edge(self, vertex1: str, vertex2: str, data: Optional[Any]=None) -> None:
        """
        Adds an edge to the graph
        :param vertex1: vertex1 key
        :param vertex2: vertex2 key
        :param data: the data associated with the vertex
        """
        if not vertex1 in self._graph or not vertex2 in self._graph:
            raise ValueError("The vertexes do not exist")
        self._graph[vertex1]['neighbors'][vertex2] = data

    def get_neighbors(self, vertex) -> List[str]:
        """
        Get the list of vertex neighbors
        :param vertex: the vertex to query
        :return: the list of neighbor vertexes
        """
        if vertex in self._graph:
            return list(self._graph[vertex]['neighbors'].keys())
        else:
            return []

    def get_vertex_data(self, vertex: str) -> Optional[Any]:
        """
        Gets  vertex associated data
        :param vertex: the vertex name
        :return: the vertex data
        """
        if self.vertex_exists(vertex):
            return self._graph[vertex]['data']
        else:
            return None

    def get_edge_data(self, vertex1: str, vertex2: str) -> Optional[Any]:
        """
        Gets the vertexes edge data
        :param vertex1: the vertex1 name
        :param vertex2: the vertex2 name
        :return: vertexes edge data
        """
        if self.edge_exists(vertex1, vertex2):
            return self._graph[vertex1]['neighbors'][vertex2]
        raise ValueError("The edge does not exist")

    def print_graph(self) -> None:
        """
        Prints the graph
        """
        for vertex, data in self._graph.items():
            print("Vertex:", vertex)
            print("Data:", data['data'])
            print("Neighbors:", data['neighbors'])
            print("")

    def vertex_exists(self, vertex: str) -> bool:
        """
        If contains a vertex
        :param vertex: the vertex name
        :return: boolean
        """
        return vertex in self._graph

    def edge_exists(self, vertex1: str, vertex2: str) -> bool:
        """
        If contains an edge
        :param vertex1: the vertex1 name
        :param vertex2: the vertex2 name
        :return: boolean
        """
        return vertex1 in self._graph and vertex2 in self._graph[vertex1]['neighbors']

In [2]:
graph = Graph()

In [3]:
from tqdm import tqdm

index_title_dict = {}

with open("vertexes.txt", "r", encoding="utf8") as file:
    for line in file:
        index = int(line.split(" ")[0])
        title = " ".join(line[:-1].split(" ")[1:])
        index_title_dict[index] = title
        graph.add_vertex(title)

with open("edges.txt", "r", encoding="utf8") as file:
    for line in tqdm(file, desc="Loading edges", total=101409330):
        art1 = int(line[:-1].split(" ")[0])
        art2 = int(line[:-1].split(" ")[1])
        q = int(line[:-1].split(" ")[2])
        if art1 in index_title_dict and art2 in index_title_dict:
            graph.add_edge(index_title_dict[art1], index_title_dict[art2], q)

del index_title_dict

Loading edges: 100%|█████████▉| 101409095/101409330 [03:18<00:00, 509765.23it/s]


In [4]:
#construir el grafo no dirigido subyacente

def build_undirected_graph(graph: Graph) -> Graph:
    """
    Builds the undirected graph
    :param graph: the graph
    :return: the undirected graph
    """
    undirected_graph = Graph()
    for vertex in graph._graph:
        undirected_graph.add_vertex(vertex)
    for vertex in tqdm(graph._graph, desc="Building undirected graph"):
        for neighbor in graph.get_neighbors(vertex):
            undirected_graph.add_edge(vertex, neighbor, graph.get_edge_data(vertex, neighbor))
            undirected_graph.add_edge(neighbor, vertex, graph.get_edge_data(vertex, neighbor))
    return undirected_graph

undirected_graph = build_undirected_graph(graph)



Building undirected graph: 100%|██████████| 3958452/3958452 [03:04<00:00, 21451.15it/s]


In [5]:
del graph

In [6]:
from collections import deque

def count_connected_components_bfs(graph: Graph) -> int:
    visited = set()  
    components = 0  

    def bfs(start_vertex):
        queue = deque([start_vertex])
        while queue:
            current = queue.popleft()
            if current not in visited:
                visited.add(current)
                for neighbor in graph.get_neighbors(current):
                    if neighbor not in visited:
                        queue.append(neighbor)

    for vertex in tqdm(graph._graph.keys(), desc="Counting components with BFS"):
        if vertex not in visited:
            components += 1
            bfs(vertex)

    return components

print(count_connected_components_bfs(undirected_graph))

del undirected_graph

Counting components with BFS: 100%|██████████| 3958452/3958452 [01:00<00:00, 65681.10it/s] 


32665


In [7]:
graph = Graph()

index_title_dict = {}

with open("vertexes.txt", "r", encoding="utf8") as file:
    for line in file:
        index = int(line.split(" ")[0])
        title = " ".join(line[:-1].split(" ")[1:])
        index_title_dict[index] = title
        graph.add_vertex(title)

with open("edges.txt", "r", encoding="utf8") as file:
    for line in tqdm(file, desc="Loading edges", total=101409330):
        art1 = int(line[:-1].split(" ")[0])
        art2 = int(line[:-1].split(" ")[1])
        q = int(line[:-1].split(" ")[2])
        if art1 in index_title_dict and art2 in index_title_dict:
            graph.add_edge(index_title_dict[art1], index_title_dict[art2], q)

del index_title_dict

Loading edges: 100%|█████████▉| 101409095/101409330 [03:22<00:00, 501691.75it/s]


In [22]:
from collections import deque

def find_shortest_path(graph, start, end):
    if start == end:
        return [0, [start]]

    dist = 0
    visited = set()
    queue = deque()
    anterior = {start: None}  
    
    queue.append((start, dist))
    
    while queue:
        current, dist = queue.popleft()
        
        if current == end:
            path = []
            while current:
                path.append(current)
                current = anterior.get(current)
            return [len(path) - 1, path[::-1]]  
        
        if current in visited:
            continue
        visited.add(current)
        
        for neighbor in graph.get_neighbors(current):
            if neighbor not in visited:
                queue.append((neighbor, dist + 1))
                if neighbor not in anterior:
                    anterior[neighbor] = current
        return []

min_path = find_shortest_path(graph, "Samsung Galaxy J7 Prime", "Guillermo Francella")

In [23]:
print(min_path)

[5, ['Samsung Galaxy J7 Prime', 'Android', 'GNU General Public License', '25 de febrero', '14 de febrero', 'Guillermo Francella']]


In [24]:
from collections import deque

def find_most_least_original_paths(graph: Graph, start: str, end: str) :
    min_paths = []
    min_length = float('inf')
    
    queue = deque([(start, [start], 0)])
    
    levels = {start: 0}
    
    while queue:
        current, path, current_weight = queue.popleft()
        
        if len(path) > min_length:
            break
        
        if current == end:
            if len(path) <= min_length:
                min_length = len(path)
                min_paths.append((path, current_weight / (len(path) - 1)))
            continue
        
        for neighbor in graph.get_neighbors(current):
            try:
                edge_weight = graph.get_edge_data(current, neighbor)
            except:
                edge_weight = 1  
            
            if neighbor not in levels or levels[neighbor] == levels.get(current, 0) + 1:
                new_path = path + [neighbor]
                new_weight = current_weight + edge_weight
                
                queue.append((neighbor, new_path, new_weight))
                levels[neighbor] = levels.get(current, 0) + 1
    
    if not min_paths:
        return None
    
    min_paths.sort(key=lambda x: x[1])
    
    return {
        'most_original_path': min_paths[0][0],  
        'least_original_path': min_paths[-1][0],  
        'most_original_weight': min_paths[0][1],
        'least_original_weight': min_paths[-1][1]
    }

def demo_originalidad(graph):
    start = "Samsung Galaxy J7 Prime"
    end = "Guillermo Francella"
    
    resultado = find_most_least_original_paths(graph, start, end)
    
    if resultado:
        print("Camino más original:")
        print(" -> ".join(resultado['most_original_path']))
        print(f"Peso promedio: {resultado['most_original_weight']}")
        
        print("\nCamino menos original:")
        print(" -> ".join(resultado['least_original_path']))
        print(f"Peso promedio: {resultado['least_original_weight']}")
    else:
        print("No se encontraron caminos")

demo_originalidad(graph)

Camino más original:
Samsung Galaxy J7 Prime -> Android -> GNU General Public License -> 25 de febrero -> 14 de febrero -> Guillermo Francella
Peso promedio: 1.0

Camino menos original:
Samsung Galaxy J7 Prime -> 4G -> Uruguay -> Argentina -> El secreto de sus ojos -> Guillermo Francella
Peso promedio: 6.4
